# Web Scraping: Estados financieros emitidos (SUSESO)

El siguiente código realiza una revisión y extracción del código fuente de la página de SUSESO. Los requisitos para poder ejecutar este código son:

 - **Instalar librerías:**
  - pandas
  - scrapy
  - requests
  - datetime
  - wget
 - **Tener creadas las carpetas para almacenar los archivos.**
  - En el **código 15** está la ruta en donde se almacena la información, las cuales deben ser cambiadas por quien ejecute el código para que referencien a su propio equipo.
  
---
 - **Fecha de creación: 07.10.2020 (IIC)** 

## Importar librerías

In [1]:
import pandas as pd
from scrapy import Selector
import requests
from datetime import datetime, timedelta
import wget

## Utilidades

In [2]:
def val(row):
    if row['Mes'] == 'Enero':
        val = 1
    elif row['Mes'] == 'Febrero':
        val = 2
    elif row['Mes'] == 'Marzo':
        val = 3
    elif row['Mes'] == 'Abril':
        val = 4
    elif row['Mes'] == 'Mayo':
        val = 5
    elif row['Mes'] == 'Junio':
        val = 6
    elif row['Mes'] == 'Julio':
        val = 7
    elif row['Mes'] == 'Agosto':
        val = 8
    elif row['Mes'] == 'Septiembre':
        val = 9
    elif row['Mes'] == 'Octubre':
        val = 10
    elif row['Mes'] == 'Noviembre':
        val = 11
    elif row['Mes'] == 'Diciembre':
        val = 12
    else:
        val = 'error'
    return val

## Cargar código html a utilizar

In [3]:
html = requests.get('https://www.suseso.cl/609/w3-propertyvalue-10342.html').content

### Crear selectors para después buscar la información

In [4]:
sel = Selector(text = html)

## Búsqueda de información

### Lista periodos

En la siguiente línea de código se busca el tag "td" con la clase *fecha periodo* y luego se busca el tag "span" que contenga una clase en particular en todo el código y se extrae el texto asociado.

In [5]:
anno = sel.xpath('//td[contains(@class,"fecha periodo")]/span[contains(@class,"pv-branch pnid-545 cid-501")]/span[1]/text()').extract()
mes = sel.xpath('//td[contains(@class,"fecha periodo")]/span[contains(@class,"pv-branch pnid-545 cid-501")]/span[2]/text()').extract()

### Lista tipo de informe

En la siguiente línea de código se busca el tag "span" que contenga una clase en particular en todo el código y se extrae el texto asociado.

In [6]:
informe = sel.xpath('//span[contains(@class,"pv-branch pnid-500 cid-501")]/span/text()').extract()

### Lista títulos

En la siguiente línea de código se busca el tag td" que contenga una clase en particular en todo el código y se extrae el texto asociado.

In [7]:
titulo = sel.xpath('//td[contains(@class,"titulo aid")]/a/text()').extract()
titulo_estatuto = titulo.pop()

### Lista con links

En la siguiente línea de código se busca en todos los tag td que tengan como "hijo" un tag span con la clase "td_20_porciento cid-19" y este a su vez, tenga asociado un hipervículo el cual se extrae.

In [8]:
links = sel.xpath('//td[contains(@class,"text-center cid-509 aid")]/a/@href').extract()
link_estuto = links.pop()

In [9]:
urls = []

for i in links:
    url = 'https://www.suseso.cl/609/' + i
    urls.append(url)

## Almacenar información obtenida y agregar formatos

In [10]:
df_eeff = pd.DataFrame({'Año' : anno, 'Mes' : mes, 'Tipo Informe' : informe, 'Título' : titulo, 'Link': urls})

#Creamos nueva columna con meses en numeros
df_eeff['mes_n'] =  df_eeff.apply(val, axis=1).astype('string')

#Creamos nueva columna auxiliar para filtrar novedades luego
df_eeff['aux'] = df_eeff['Año'] + df_eeff['mes_n'] + str(30)

#Aplicamos formato fecha a columna "aux"
df_eeff['aux'] = pd.to_datetime(df_eeff['aux'], format='%Y%m%d')

#Quitamos ":" de los títulos y reemplazamos por " "
df_eeff['Título'] = df_eeff['Título'].str.replace(':',' ')

## Filtrar novedades

In [11]:
novedad = datetime.now() - timedelta(days=180)

In [12]:
df_eeff_novedad = df_eeff.loc[(df_eeff['aux']>=novedad)]
df_eeff_novedad

,Año,Mes,Tipo Informe,Título,Link,mes_n,aux
52,2020,Junio,Informe consolidado,ACHS Estados financieros consolidados al 30 d...,https://www.suseso.cl/609/articles-600647_arch...,6,2020-06-30
53,2020,Junio,Informe individual,ACHS Estados financieros individuales al 30 d...,https://www.suseso.cl/609/articles-600648_arch...,6,2020-06-30


In [13]:
nombre_archivo = df_eeff_novedad['Año'] + df_eeff_novedad['Mes'] + df_eeff_novedad['Tipo Informe'] + '.pdf'
nombre_archivo = nombre_archivo.str.replace(' ','')

In [14]:
urls_novedad = df_eeff_novedad['Link']

## Descargar novedades

### Rutas para almacenar la información

In [15]:
paths = []

for i in nombre_archivo:
    # La ruta escrita entre '' debe ser cambiada por quien ejecute el código.
    path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Descargas_RPA/SUSESO_EEFF/' + i
    paths.append(path)

### Descargar archivos

In [16]:
i = 0

for url in urls_novedad:
    wget.download(url,paths[i])
    i = i + 1

### Guardar DF históricos en Excel

In [17]:
# Eliminamos columnas auxiliares
df_eeff = df_eeff.drop(['mes_n','aux'], axis=1)

In [18]:
nombre_archivo = 'EEFF_emitidos.xlsx'
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')

df_eeff.to_excel(writer, sheet_name='Estados Financieros')

writer.save()